**Ensemble Learning**
- **CNN**.
- **RNN** implemented using a *LSTM*.

It can uses also a *random seed* to shuffle the data and use a different *training*, *validation* and *test* set respect to the ones used by ***ViraMiner***.

In [ ]:
"""
Main libraries
"""

import torch as th
import torch.nn as nn
import numpy as np
import pandas as pd
import tensorflow as tf

import random

In [ ]:
"""
For the ensamble learning
"""
from sklearn.svm import SVC

In [ ]:
"""
Only for the metrics analysis
"""
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

The **Onehot Encoding** that transform a *DNA sequence* to a vector of zeros and ones. It is useful expectially for the **CNN**.

In [ ]:
"""
Functions for the onehot encoding
"""

def onehot_encoder(dataset):
    """
    Function that encodes a DNA dataset into a onehot encoding dataset.
    """
    onehot_dataset = [dna_onehot_encoder(dna_string) for dna_string in dataset]
    onehot_dataset_numpy = np.array(onehot_dataset)

    return onehot_dataset_numpy


def dna_onehot_encoder(dna_sequence):
    """
    Function that encodes a single DNA string into a onehot encoding string.
    """
    onehot_dict = {
        'A' : [1, 0, 0, 0],
        'C' : [0, 1, 0, 0],
        'G' : [0, 0, 1, 0],
        'T' : [0, 0, 0, 1]
    }
    encoder = [onehot_dict[nuc] for nuc in dna_sequence]

    return encoder

In [ ]:
class ViraMinerDataset (Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def one_hot_encode(self, sequence):
        mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
        seq_length = len(sequence)
        one_hot = th.zeros((seq_length, 4), dtype=th.float32)
        for i, base in enumerate(sequence):
            one_hot[i, mapping[base]] = 1

        return one_hot

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        one_hot_sequence = self.one_hot_encode(sequence)
        return one_hot_sequence, label

The **CNN** model: direct from the file *CNN.ipynb* avaiable in the GitHub repository.

In [ ]:
"""
!- CNN Model
"""

class ConvNet(nn.Module):
    # We can use a differnet pool for each layer
    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv1d(300, 200, kernel_size=2, padding=1),
            nn.BatchNorm1d(200),
            nn.PReLU(),
            nn.AvgPool1d(2),
            nn.Dropout1d(0.45)
        )

        # I remove random connection to help the convergency
        self.drop_out = nn.Dropout()

        self.layer2 = nn.Sequential(
            nn.Conv1d(200, 100, kernel_size=2, padding=1),
            nn.BatchNorm1d(100),
            nn.PReLU(),
            nn.AvgPool1d(2),
            nn.Dropout1d(0.45)
        )

        # I remove random connection to help the convergency
        self.drop_out = nn.Dropout()

        self.layer3 = nn.Sequential(
            nn.Conv1d(100, 75, kernel_size=2, padding=1),
            nn.BatchNorm1d(75),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout1d(0.3)
        )

        self.layer4 = nn.Sequential(
            nn.Conv1d(75, 50, kernel_size = 2, padding = 1),
            nn.BatchNorm1d(50),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.3)
        )


        self.layer5 = nn.Sequential(
            nn.Conv1d(50, 32, kernel_size=2, padding=1),
            nn.BatchNorm1d(32),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout1d(0.3)
        )

        self.linear1 = nn.Linear(32, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.layer1(x)
        # print("Layer 1: ", x.shape)
        x = self.layer2(x)
        # print("Layer 2: ", x.shape)
        x = self.layer3(x)
        # print("Layer 3: ", x.shape)
        x = self.layer4(x)
        # print("Layer 4: ", x.shape)
        x = self.layer5(x)
        # print("Layer 5: ", x.shape)
        # Flatten the output for the linear layer
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.linear2(x)
        # print("Linear: ", x.shape)
        return x

The **RNN** model: direct from the file *LSTM.ipynb* avaiable in the GitHub repository. We choose the *LSTM* over the *GRU* because it obtained an **higher** precision on the dataset and also is **less** computationally demanding.

In [ ]:
class LSTM_Net (nn.Module):
    def __init__ (self):
        super(LSTM_Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, num_layers=2, batch_first=True, dropout=0.5)

        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)

        x = x[:, -1, :]
        x = x.contiguous().view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

**MAIN**

Training phase

In [ ]:
"""
!- MAIN
"""

# Set the device to be used (GPU or CPU)
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print("Device: ", device)

# Read the input from the cvc file
"""
If run in local machine:

data_dir = os.path.abspath(os.path.join(os.getcwd(), 'data'))
train_data_path = os.path.join(data_dir, 'fullset_train.csv')
val_data_path = os.path.join(data_dir, 'fullset_validation.csv')
test_data_path = os.path.join(data_dir, 'fullset_test.csv')
"""

rel_path_train = '/content/fullset_train.csv'
rel_path_val = '/content/fullset_validation.csv'
rel_path_test = '/content/fullset_test.csv'

# Training Set

# Read the input from the csv file
train_csv = pd.read_csv(rel_path_train, sep=",")
# Drop the NaN values
train_csv = train_csv.dropna()
# Describe the data
print(train_csv.describe())

# Get the data from the csv file
train_data = train_csv.values
# m = number of input samples
m = train_data.shape[0]

X_train = train_data[:m,1]
Y_train = train_data[:m,2].astype(np.int32)

# OneHot encoding for the training data
print("Start onehot encoding for the training data")
X_train = onehot_encoder(X_train)

# Convert the data to a tensor
X_train = th.from_numpy(X_train).to(device)
Y_train = th.tensor(Y_train).to(device)

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

# Free memory
del train_csv, train_data, m

# Validation Set

# Read the input from the csv file
val_csv = pd.read_csv(rel_path_val, sep=",")
# Drop the NaN values
val_csv = val_csv.dropna()
# Describe the data
print(val_csv.describe())

val_data = val_csv.values
# m = number of input samples
m = val_data.shape[0]

X_val = val_data[:m,1]
Y_val = val_data[:m,2].astype(np.int32)

# OneHot encoding for the validation data
print("Start onehot encoding for the validation data")
X_val = onehot_encoder(X_val)

X_val = th.from_numpy(X_val).to(device)
Y_val = th.tensor(Y_val).to(device)

print("X_val shape", X_val.shape)
print("Y_val shape", Y_val.shape)

# Free memory
del val_csv, val_data, m

# Test

# Read the input from the csv file
test_csv = pd.read_csv(rel_path_test, sep=",")
# Drop the NaN values
test_csv = test_csv.dropna()
# Describe the data
print(test_csv.describe())

test_data = test_csv.values
# m = number of input samples
m = test_data.shape[0]

X_test = test_data[:m,1]
Y_test = test_data[:m,2].astype(np.int32)

# OneHot encoding for the test data
print("Start onehot encoding for the test data")
X_test = onehot_encoder(X_test)

X_test = th.from_numpy(X_test).to(device)
Y_test = th.tensor(Y_test).to(device)

print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)

# Free memory
del test_csv, test_data, m

**Shuffle of the data** *(optional)* using a random seed.

In [ ]:
"""
I merge the three tensor array to a big one and then, after a shuffle, I split the data into:
  - Training: 211238 data
  - Validation: 26404 data
  - Test: 26404 data

  Random seed: 47 (the most used)
"""

# Merge the arrays

print("Shape of X_train: ", X_train.shape)
print("Shape of X_val: ", X_val.shape)
print("Shape of X_test: ", X_test.shape)

X_data = th.concat((X_train, X_val), axis = 0).to(device)
X_data = th.concat((X_data, X_test), axis = 0).to(device)

print("Shape of data: ", X_data.shape)

del X_train, X_val, X_test

print("Shape of Y_train: ", Y_train.shape)
print("Shape of Y_val: ", Y_val.shape)
print("Shape of Y_test: ", Y_test.shape)

Y_data = th.concat((Y_train, Y_val), axis = 0).to(device)
Y_data = th.concat((Y_data, Y_test), axis = 0).to(device)

print("Shape of data: ", Y_data.shape)

del Y_train, Y_val, Y_test

# Random Seed
random_seed = 47
th.manual_seed(random_seed)

X_index_shuffle = th.randperm(X_data.size(0))
X_data_shuffled = X_data[X_index_shuffle]

Y_index_shuffle = th.randperm(Y_data.size(0))
Y_data_shuffled = Y_data[Y_index_shuffle]

print("X_data_shuffled shape: ", X_data_shuffled.shape)
print("Y_data_shuffled shape: ", Y_data_shuffled.shape)

# Split data into Training, Validation and Test

X_train = X_data_shuffled[:211238].to(device)
Y_train = Y_data_shuffled[:211238].to(device)
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

X_val = X_data_shuffled[211238:237642]
Y_val = Y_data_shuffled[211238:237642]
print("X_val shape: ", X_val.shape)
print("Y_val shape: ", Y_val.shape)

X_test = X_data_shuffled[237642:].to(device)
Y_test = Y_data_shuffled[237642:].to(device)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

*Train* of the **CNN** using the *Validation* set to check the performance.

In [ ]:
print("Start training the model")

model_CNN = ConvNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = th.optim.AdamW(model_CNN.parameters(), lr=0.001)
scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

# Early stopping parameters
patience = 3  # CHECK THE VALUE!!
best_val_loss = float('inf')
counter = 0

# Training the model
num_epochs = 100  # Hope to reach convergence before 100 epoches
train_loss_CNN, val_loss_CNN = [], []

for epoch in range(num_epochs):
    model_CNN.train()
    X_train = X_train.float()
    Y_train = Y_train.long()
    outputs = model_CNN(X_train.to(device))
    loss = criterion(outputs, Y_train.to(device))
    train_loss_CNN.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation the model
    model_CNN.eval()
    with th.no_grad():
        X_val = X_val.float()
        Y_val = Y_val.long()
        val_outputs = model_CNN(X_val.to(device))
        val_loss = criterion(val_outputs, Y_val.to(device))
        val_loss_CNN.append(val_loss.item())
        scheduler.step(val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    # Check for early stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print("End training the model")

**(OPTIONAL):** *Test* results for the **CNN**.

In [ ]:
print("Start testing the CNN")

X_test = X_test.to(device).float()

# Model ready for the evaluation
model_CNN.eval()

# Testing the model
with th.no_grad():
    outputs = model_CNN(X_test)
    _, predicted = th.max(outputs, 1)

# Convert the tensor to use scikit learn metrics
y_true = Y_test.cpu().numpy()
y_pred = predicted.cpu().numpy()

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')

print("End testing the CNN")

*Train* of the **RNN** using the *Validation* set to check the performance.

In [ ]:
# Model, loss function and optimizer
model_LSTM = LSTM_Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model_LSTM.parameters(), lr=0.001)

# Training the model
num_epochs = 7
train_loss_RNN, val_loss_RNN = [], []

for epoch in range(num_epochs):
    model_LSTM.train()
    running_loss = 0.0

    X_train = X_train.float()
    Y_train = Y_train.long()

    # Forward pass
    outputs = model_LSTM(X_train.to(device))
    loss = criterion(outputs, Y_train.to(device))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    # Validation
    model_LSTM.eval()
    with th.no_grad():
      X_val = X_val.float()
      Y_val = Y_val.long()
      X_val = X_val.to(device)
      Y_val = Y_val.to(device)

      val_outputs = model_CNN(X_val.to(device))
      val_loss = criterion(val_outputs, Y_val.to(device))
      val_loss_CNN.append(val_loss.item())
      scheduler.step(val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

**(OPTIONAL):** *Test* results forthe **RNN**.

In [ ]:
print("Start testing the LSTM")

X_test = X_test.to(device).float()

# Model ready for the evaluation
model_LSTM.eval()

# Testing the model
with th.no_grad():
    outputs = model_LSTM(X_test)
    _, predicted = th.max(outputs, 1)

# Convert the tensor to use scikit learn metrics
y_true = Y_test.cpu().numpy()
y_pred = predicted.cpu().numpy()

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')

print("End testing the LSTM")

Implementation of the **Ensemble Learning** using an **SVM** (stacking ensemble learing).

In [ ]:
"""
Prepare the data for the final evaluation using DataLoader
"""

"""
Function that, given a model and a dataset, it returns all the predictions (outputs)
"""
def get_predictions(model_pred, dataset):
    model_pred.eval()
    all_preds = []
    with th.no_grad():
        for inputs, _ in dataset:
            outputs = model_pred(inputs)
            all_preds.append(outputs.cpu().numpy())
    return np.concatenate(all_preds)

# Prepare the prediction for the training model
CNN_train_preds = get_predictions(model_CNN, X_train)
LSTM_train_preds = get_predictions(model_LSTM, X_train)

# Concatenate the two inputs
X_train_meta = np.hstack((CNN_train_preds, LSTM_train_preds))

In [ ]:
"""
An SVM takes in input the results from the CNN and RNN and then it preforms the final classification.

I have to use skitlearn to do the ensable learing because the library is only avaiable on it!
"""

# We use a polynomial SVM for the final classification
model_SVM = SVC(kernel = 'poly', probability = True)

model_SVM.fit(X_train_meta, Y_train.ravel())

**Test** of the performance.

In [ ]:
print("Start testing the final model")

# I convert the data (if needed)
y_true = y_test.cpu().numpy() if isinstance(y_test, th.Tensor) else y_test

# Testing the meta-model: SVM
y_pred = model_SVM.predict(X_test_meta)

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')

print("End testing the final model")